In [3]:
import torch 
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))

X = torch.rand(2,20)
net(X)

tensor([[-0.0952, -0.0438,  0.0226,  0.1597,  0.0172,  0.2618,  0.2272,  0.0184,
         -0.1736,  0.1316],
        [-0.1202, -0.0567,  0.1088,  0.1786,  0.1001,  0.2665,  0.2768,  0.0044,
         -0.2042,  0.2575]], grad_fn=<AddmmBackward0>)

In [5]:
class MLP(nn.Module):
    '''用模型参数声明层。这里，我们声明两个全连接层'''
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20,256) #隐藏层
        self.out = nn.Linear(256,10) #输出层

    #定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self,X):
        #注意，这里我们使用Relu，在nn.functional模块中定义
        return self.out(F.relu(self.hidden(X)))

In [6]:
net = MLP()
net(X)

tensor([[ 0.0338,  0.1509, -0.0837, -0.0327, -0.2092, -0.1782, -0.1814,  0.0559,
         -0.0792,  0.1921],
        [ 0.0705,  0.2134,  0.1066, -0.1055, -0.2561, -0.2021, -0.2895, -0.0174,
         -0.0243,  0.1832]], grad_fn=<AddmmBackward0>)

In [7]:
class MySequential(nn.Module):
    def __init__(self,*args) -> None:
        super().__init__()
        for idx,module in enumerate(args):
            # module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module
    
    def forward(self,X):
        #OrderedDict保证了按照成员添加的顺序遍历它
        for block in self._modules.values():
            X = block(X)
        return X

In [8]:
net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[ 0.0574, -0.0932, -0.0970, -0.2817, -0.3089,  0.1463, -0.0674,  0.2907,
          0.0485, -0.0239],
        [ 0.1492, -0.2238, -0.1621, -0.2582, -0.3656,  0.0633, -0.0526,  0.1138,
         -0.0392,  0.1329]], grad_fn=<AddmmBackward0>)

In [9]:
class FixedHiddenMLP(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.rand_weight = torch.rand((20,20),requires_grad=False)
        self.linear = nn.Linear(20,20)
    
    def forward(self,X):
        X = self.linear(X)
        #使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X,self.rand_weight) + 1)
        #复用全连接层
        X = self.linear(X)
        #控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [12]:
net = FixedHiddenMLP()
net(X)

tensor(-0.2231, grad_fn=<SumBackward0>)

In [13]:
class NestMLP(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64),nn.ReLU(),
                                 nn.Linear(64,32),nn.ReLU())
        self.linear = nn.Linear(32,16)
        
    def forward(self,X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(),nn.Linear(16,20),FixedHiddenMLP())
chimera(X)

tensor(-0.2898, grad_fn=<SumBackward0>)